# Let's go PRO!

Advanced RAG Techniques!

Let's start by digging into ingest:

1. No LangChain! Just native for maximum flexibility
2. Let's use an LLM to divide up chunks in a sensible way
3. Let's use the best chunk size and encoder from yesterday
4. Let's also have the LLM rewrite chunks in a way that's most useful ("document pre-processing")

In [175]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
from litellm import completion
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


load_dotenv(override=True)

MODEL = "gpt-4.1-nano"

DB_NAME = "preprocessed_db"
collection_name = "docs"
embedding_model = "text-embedding-3-large"
KNOWLEDGE_BASE_PATH = Path("knowledge-base")
AVERAGE_CHUNK_SIZE = 500

openai = OpenAI()

In [46]:
# Inspired by LangChain's Document - let's have something similar

class Result(BaseModel):
    page_content: str
    metadata: dict

In [47]:
# A class to perfectly represent a chunk

class Chunk(BaseModel):
    headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
    summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
    original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

    def as_result(self, document):
        metadata = {"source": document["source"], "type": document["type"]}
        return Result(page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,metadata=metadata)


class Chunks(BaseModel):
    chunks: list[Chunk]

## Three steps:

1. Fetch documents from the knowledge base, like LangChain did
2. Call an LLM to turn documents into Chunks
3. Store the Chunks in Chroma

That's it!

### Let's start with Step 1

In [48]:
def fetch_documents():
    """A homemade version of the LangChain DirectoryLoader"""

    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({"type": doc_type, "source": file.as_posix(), "text": f.read()})

    print(f"Loaded {len(documents)} documents")
    return documents

In [49]:
documents = fetch_documents()

Loaded 76 documents


### Donezo! On to Step 2 - make the chunks

In [50]:
def make_prompt(document):
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
"""

In [51]:
print(make_prompt(documents[0]))


You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: products
The document has been retrieved from: knowledge-base/products/Rellm.md

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into 8 chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

# Product Summary

# Rellm: AI-Powered Enterprise 

In [52]:
def make_messages(document):
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [53]:
make_messages(documents[0])

[{'role': 'user',
  'content': "\nYou take a document and you split the document into overlapping chunks for a KnowledgeBase.\n\nThe document is from the shared drive of a company called Insurellm.\nThe document is of type: products\nThe document has been retrieved from: knowledge-base/products/Rellm.md\n\nA chatbot will use these chunks to answer questions about the company.\nYou should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.\nThis document should probably be split into 8 chunks, but you can have more or less as appropriate.\nThere should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.\n\nFor each chunk, you should provide a headline, a summary, and the original text of the chunk.\nTogether your chunks should represent the entire document with overlap.\n\nHere is the document:\n\n#

In [54]:
def process_document(document):
    messages = make_messages(document)
    response = completion(model=MODEL, messages=messages, response_format=Chunks)
    reply = response.choices[0].message.content
    doc_as_chunks = Chunks.model_validate_json(reply).chunks
    return [chunk.as_result(document) for chunk in doc_as_chunks]

In [55]:
process_document(documents[0])

/Users/frankbarnhouse/GitHub Repos/llm_engineering/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='{"chunks...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])



[Result(page_content='Product Overview of Rellm\n\nRellm is an innovative enterprise reinsurance platform developed by Insurellm that leverages artificial intelligence to improve risk management, decision-making, and operational efficiency in the reinsurance industry.\n\n# Product Summary\n\n# Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Summary\n\nRellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility.', metadata={'source': 'knowledge-base/products/Rellm.md', 'type': 'products'}),
 Result(page_content="Core Features of R

In [56]:
def create_chunks(documents):
    chunks = []
    # tqdm is a progress bar - very cool :-)
    for doc in tqdm(documents):
        chunks.extend(process_document(doc))
    return chunks

In [57]:
chunks = create_chunks(documents)

100%|██████████| 76/76 [12:38<00:00,  9.98s/it]


In [58]:
print(len(chunks))

418


### Well that was easy! If a bit slow.

In the python module version, I sneakily use the multi-processing Pool to run this in parallel,
but if you get a Rate Limit Error you can turn this off in the code.

### Finally, Step 3 - save the embeddings

In [59]:
def create_embeddings(chunks):
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]
    emb = openai.embeddings.create(model=embedding_model, input=texts).data
    vectors = [e.embedding for e in emb]

    collection = chroma.get_or_create_collection(collection_name)

    ids = [str(i) for i in range(len(chunks))]
    metas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")

In [60]:
create_embeddings(chunks)

Vectorstore created with 418 documents


# Nothing more to do here... right?

Wait! Didja think I'd forget??

In [61]:
chroma = PersistentClient(path=DB_NAME)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [62]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [19]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

## And now - let's build an Advanced RAG!

We will use these techniques:

1. Reranking - reorder the rank results
2. Query re-writing

In [63]:
class RankOrder(BaseModel):
    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [65]:
def rerank(question, chunks):
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
"""
    user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
    user_prompt += "Here are the chunks:\n\n"
    for index, chunk in enumerate(chunks):
        user_prompt += f"# CHUNK ID: {index + 1}:\n\n{chunk.page_content}\n\n"
    user_prompt += "Reply only with the list of ranked chunk ids, nothing else."
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = completion(model=MODEL, messages=messages, response_format=RankOrder)
    reply = response.choices[0].message.content
    order = RankOrder.model_validate_json(reply).order
    print(order)
    return [chunks[i - 1] for i in order]

In [133]:
RETRIEVAL_K = 60

def fetch_context_unranked(question):
    query = openai.embeddings.create(model=embedding_model, input=[question]).data[0].embedding
    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))
    return chunks

In [134]:
question = "Who won the IIOTY award?"
chunks = fetch_context_unranked(question)

In [135]:
for chunk in chunks:
    print(chunk.page_content[:25]+"...")

Performance History from ...
IoT, Catastrophe Response...
Performance History (2019...
Annual Performance Histor...
Performance and Achieveme...
Career Progression at Ins...
Annual Performance Rating...
Professional Progression ...
Annual Performance Histor...
Compensation and Recognit...
Insurellm Career Progress...
Performance Ratings and T...
Performance Evaluations (...
Annual Performance Rating...
HR Initiatives, Developme...
Annual Performance Histor...
Performance Ratings and A...
HR Record Overview for Ma...
Performance Ratings and A...
Compensation and Recognit...
Career Progression at Ins...
Annual Performance Histor...
Career Progression at Ins...
Insurellm Career Progress...
Career Progression at Ins...
Nina Patel's Performance ...
Signatures and Contract C...
Insurellm Career Progress...
Scale and Client Impact o...
HR Record and Personal De...
Educational Background, R...
Performance History and R...
Salary and HR Notes

This...
Annual Performance Rating...
Career Progres

In [137]:
reranked = rerank(question, chunks)

[1, 16, 18, 3, 4, 6, 8, 11, 13, 17, 19, 20, 22, 23, 24, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


In [138]:
for chunk in reranked:
    print(chunk.page_content[:25]+"...")

Performance History from ...
Annual Performance Histor...
HR Record Overview for Ma...
Performance History (2019...
Annual Performance Histor...
Career Progression at Ins...
Professional Progression ...
Insurellm Career Progress...
Performance Evaluations (...
Performance Ratings and A...
Performance Ratings and A...
Compensation and Recognit...
Annual Performance Histor...
Career Progression at Ins...
Insurellm Career Progress...
Signatures and Contract C...
Insurellm Career Progress...
Scale and Client Impact o...
Educational Background, R...
Performance History and R...
Salary and HR Notes

This...
Annual Performance Rating...
Career Progression at Ins...
Introduction and Founding...
Annual Performance Histor...
Signatures and Closing Re...
Performance History (Part...
Annual Performance Histor...
Performance and Compensat...
Career Progression at Ins...
Career Progression at Ins...
Additional HR Notes and D...
Compensation and HR Notes...
Career Progression at Ins...
Personal and R

In [163]:
question = "Who went to Manchester University?"
RETRIEVAL_K = 200
chunks = fetch_context_unranked(question)
for index, c in enumerate(chunks):
    if "manchester" in c.page_content.lower():
        print(index)

In [164]:
reranked = rerank(question, chunks)

[3, 74, 8, 45, 191, 98, 173, 43, 24, 61, 148, 44, 10, 112, 70, 34, 15, 113, 51, 84, 117, 76, 75, 140, 11, 76, 26, 79, 133, 168, 149, 27, 81, 132, 144, 55, 35, 64, 100, 122, 146, 156, 158, 96, 155, 54, 46, 52, 158, 156, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 156, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 54, 46, 52, 158, 26, 79, 25, 138, 101, 30, 66, 36, 106, 16, 5, 9, 21, 57, 63, 80, 111, 139, 87, 77, 68, 4, 87, 1, 20, 114, 143, 13, 156, 158, 155, 96, 30, 66, 106, 5, 9, 21, 57, 63, 80, 111, 139, 87, 77, 68, 4, 87, 1, 20, 114, 143, 13, 156, 158, 155, 96, 30, 66, 106, 5, 9, 21, 57, 63, 80, 111, 139, 87, 77, 68, 4, 87, 1, 20, 114, 143, 13, 156, 158, 155, 96, 30, 66, 106, 5, 9, 21, 57, 63, 80, 111, 139, 87, 77, 68, 4, 87, 1, 20, 114, 143, 13, 156, 158, 155, 96, 30, 66, 106, 5,

In [165]:
for index, c in enumerate(reranked):
    if "manchester" in c.page_content.lower():
        print(index)

In [166]:
reranked[0].page_content

"Additional HR Notes and Qualifications for Marcus Johnson\n\nHighlights Marcus Johnson's education, certifications, awards, mentorship activities, and professional strengths as noted in HR documents.\n\n## Other HR Notes\n- **Education:** BA in Business Administration from NYU Stern School of Business\n- **Certifications:** Certified Customer Success Manager (CCSM), Salesforce Administrator\n- **Recognition:** Customer Champion Award 2023 for highest NPS scores\n- **Mentorship:** Actively mentors new Customer Success Associates\n- **Feedback:** Excellent relationship builder with deep product knowledge. Strong strategic advisor to clients. Known for going above and beyond to ensure client success."

In [167]:
def fetch_context(question):
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [168]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [169]:
# In the context, include the source of the chunk

def make_rag_messages(question, history, chunks):
    context = "\n\n".join(f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks)
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

In [170]:
def rewrite_query(question, history=[]):
    """Rewrite the user's question to be a more specific question that is more likely to surface relevant content in the Knowledge Base."""
    message = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""
    response = completion(model=MODEL, messages=[{"role": "system", "content": message}])
    return response.choices[0].message.content

In [171]:
# rewrite_query("Who won the IIOTY award?", [])
rewrite_query("Who went to Manchester University?", [])

'Who attended Manchester University?'

In [172]:
def answer_question(question: str, history: list[dict] = []) -> tuple[str, list]:
    """
    Answer a question using RAG and return the answer and the retrieved context
    """
    query = rewrite_query(question, history)
    print(query)
    chunks = fetch_context(query)
    messages = make_rag_messages(question, history, chunks)
    response = completion(model=MODEL, messages=messages)
    return response.choices[0].message.content, chunks

In [96]:
answer_question("Who won the IIOTY award?", [])

Who received the IIOTY award?
[1, 2, 3, 4, 13, 21, 22, 23, 25, 27, 28, 29, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 0, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 24, 26, 30, 31, 32, 40]


('Maxine Thompson was recognized with the Insurellm IIOTY Innovator Award in 2023.',
 [Result(page_content='Additional HR Notes and Initiatives\n\nOther relevant HR information including training, awards, mentorship, and areas for professional development.\n\n## Other HR Notes\n- Maxine participated in various company-sponsored trainings related to big data technologies and cloud infrastructure.  \n- She was recognized for her contributions with the prestigious Insurellm IIOTY Innovator Award in 2023.  \n- Maxine is currently involved in the women-in-tech initiative and participates in mentorship programs to guide junior employees.  \n- Future development areas include improving her stakeholder communication skills to ensure smoother project transitions and collaboration.', metadata={'source': 'knowledge-base/employees/Maxine Thompson.md', 'type': 'employees'}),
  Result(page_content="Career Progression at Insurellm (Part 2)\n\nContinued overview of Maxine's career, from her promotion 

In [174]:
answer_question("Who went to Manchester University?", [])

Who from Insurellm attended Manchester University?


/Users/frankbarnhouse/GitHub Repos/llm_engineering/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Who from...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])



[4, 17, 72, 1, 3, 7, 155, 5, 10, 15, 42, 50, 14, 56, 164, 60, 85, 116, 162, 197, 193, 111, 116, 53, 123, 110, 136, 179, 112, 143, 27, 120, 21, 74, 125, 78, 153, 161, 186, 194, 109, 148, 66, 67, 75, 134, 83, 131, 138, 147, 169, 92, 52, 137, 172, 49, 43, 20, 28, 53, 59, 8, 29, 44, 80, 81, 143, 150, 146, 168, 174, 46, 2, 12, 41, 16, 22, 26, 37, 47, 31, 155, 155, 30, 40, 39, 81, 82, 158, 170, 171, 183, 187, 171, 96, 154, 154, 160, 165, 68, 190, 149, 191, 175, 177, 123, 132, 130, 177, 122, 33, 45, 136, 55, 55, 140, 185, 127, 129, 163, 167, 176, 178, 184, 193, 194, 195, 199, 200, 34, 36, 38, 112, 196, 179, 181, 141, 155, 185]


/Users/frankbarnhouse/GitHub Repos/llm_engineering/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning:

Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='The avai...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])



('The available information does not indicate that any Insurellm employee attended Manchester University. There is no record of an educational background linked to Manchester University in the employee profiles or other provided documents.',
 [Result(page_content='Career Progression at Insurellm and Previous Roles\n\nAvery co-founded Insurellm in 2015 and previously worked at Innovate Insurance Solutions and Edge Analytics, gaining extensive experience in insurance and analytics.\n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \nAvery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \nBefore launching Insurellm, Avery was a leading Senior Pro